# Load the data

In [1]:
import pandas as pd
train_df = pd.read_csv("../../cleaned_dataset/split_data/model_selection/train.csv")
test_df = pd.read_csv("../../cleaned_dataset/split_data/model_selection/test.csv")

# Now check the label and filter before doing anything

In [9]:
# Get all of the unique labels for train / test sets 
train_labels = list(train_df['room'].unique())
test_labels = list(test_df['room'].unique())

# Labels that appear in BOTH train and test
common_labels = list(set(train_labels) & set(test_labels))

# Now filter the train_df and test_df so that
# remove all record in train_df whose labels not in test labels 
# and remove all record in test_df whose labels not in train labels 
# therefore we can have a proper train and test df such that the labels appear 
# in train and test should be the same and vice versa
train_df = train_df[train_df['room'].isin(common_labels)].reset_index(drop=True)
test_df  = test_df[test_df['room'].isin(common_labels)].reset_index(drop=True)

In [11]:
# sanity check for label matching between train and test set
assert set(train_df['room'].unique()) == set(test_df['room'].unique())
print("Train/Test labels are now perfectly aligned")

Train/Test labels are now perfectly aligned


In [12]:
train_df.head()

,timestamp,mac address,RSSI,room
0,2023-04-10 14:21:46+09:00,6,-93,kitchen
1,2023-04-10 14:21:46+09:00,6,-93,kitchen
2,2023-04-10 14:21:46+09:00,6,-93,kitchen
3,2023-04-10 14:21:46+09:00,6,-93,kitchen
4,2023-04-10 14:21:46+09:00,6,-93,kitchen


In [13]:
test_df.head()

,timestamp,mac address,RSSI,room
0,2023-04-12 10:00:53+09:00,4,-92,cafeteria
1,2023-04-12 10:00:53+09:00,4,-96,cafeteria
2,2023-04-12 10:00:53+09:00,4,-96,cafeteria
3,2023-04-12 10:00:53+09:00,4,-96,cafeteria
4,2023-04-12 10:00:53+09:00,4,-83,cafeteria


# Feature engineering

### Firstly, create a matrix of beacon value for each records

In [14]:
# Function to add 25 beacon_1, beacon_2, ... , beacon_25
def add_beacon_features(df, num_beacons=25):
    df = df.copy()

    for i in range(1, num_beacons + 1):
        df[f'beacon_{i}'] = df['RSSI'].where(df['mac address'] == i, 0)

    return df

# Apply with our current train and test dataframe
train_df = add_beacon_features(train_df)
test_df = add_beacon_features(test_df)

In [15]:
train_df.head()

,timestamp,mac address,RSSI,room,beacon_1,beacon_2,beacon_3,beacon_4,beacon_5,beacon_6,...,beacon_16,beacon_17,beacon_18,beacon_19,beacon_20,beacon_21,beacon_22,beacon_23,beacon_24,beacon_25
0,2023-04-10 14:21:46+09:00,6,-93,kitchen,0,0,0,0,0,-93,...,0,0,0,0,0,0,0,0,0,0
1,2023-04-10 14:21:46+09:00,6,-93,kitchen,0,0,0,0,0,-93,...,0,0,0,0,0,0,0,0,0,0
2,2023-04-10 14:21:46+09:00,6,-93,kitchen,0,0,0,0,0,-93,...,0,0,0,0,0,0,0,0,0,0
3,2023-04-10 14:21:46+09:00,6,-93,kitchen,0,0,0,0,0,-93,...,0,0,0,0,0,0,0,0,0,0
4,2023-04-10 14:21:46+09:00,6,-93,kitchen,0,0,0,0,0,-93,...,0,0,0,0,0,0,0,0,0,0


In [16]:
test_df.head()

,timestamp,mac address,RSSI,room,beacon_1,beacon_2,beacon_3,beacon_4,beacon_5,beacon_6,...,beacon_16,beacon_17,beacon_18,beacon_19,beacon_20,beacon_21,beacon_22,beacon_23,beacon_24,beacon_25
0,2023-04-12 10:00:53+09:00,4,-92,cafeteria,0,0,0,-92,0,0,...,0,0,0,0,0,0,0,0,0,0
1,2023-04-12 10:00:53+09:00,4,-96,cafeteria,0,0,0,-96,0,0,...,0,0,0,0,0,0,0,0,0,0
2,2023-04-12 10:00:53+09:00,4,-96,cafeteria,0,0,0,-96,0,0,...,0,0,0,0,0,0,0,0,0,0
3,2023-04-12 10:00:53+09:00,4,-96,cafeteria,0,0,0,-96,0,0,...,0,0,0,0,0,0,0,0,0,0
4,2023-04-12 10:00:53+09:00,4,-83,cafeteria,0,0,0,-83,0,0,...,0,0,0,0,0,0,0,0,0,0


### Secondly. check the minority and majority class to know which class to be handled

In [17]:
train_df['room'].value_counts()

room
nurse station    488303
cafeteria        171492
kitchen          132819
hallway           63848
523               22169
511                8505
522                8079
cleaning           6232
506                4995
512                3773
520                3695
501                3415
517                1397
513                1152
502                 801
518                 252
Name: count, dtype: int64

In [18]:
test_df['room'].value_counts()

room
nurse station    34498
kitchen          28076
cafeteria        25229
513              12932
cleaning         10817
523               8484
520               6125
512               5635
hallway           3612
511               2042
502               1325
522                917
506                883
501                628
518                180
517                 42
Name: count, dtype: int64

- So we can see that here are the minority classes that should be handle properly, these classes are having extremely small number of records: 
    + 517 
    + 518 